Api Building


In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
import webbrowser
import time
from google.colab import drive
drive.mount('/content/drive')

historic_ether_price = pd.read_csv("/content/drive/MyDrive/Crypto/Analysis/eth-usd-max.csv") 
preprocessed_ether_address = pd.read_csv("/content/drive/MyDrive/Crypto/Analysis/Main 1to21Dec2022alladdress_morethan1txn.csv")

import sys
sys.path.insert(0,'/content/drive/MyDrive/Crypto/Analysis/ABI.py')

Mounted at /content/drive


In [22]:
"""
EVERY ETHEREUM TXN DETAILS CAN BE FOUND WITHIN:
a) ALL TXN (TXN SEND "BY PERSON OR TO PERSON" TO THE ETHEREUM BLOCKCHAIN )
b) INTERNAL TXN (ALL ETHER TXN (ETHEREUM NATIVE CURRENCY))
c) ERC20 TXN  (ERC20 TOKEN TXN (DIVISIBLE TOKEN, COULD BE GENERATED BY ANYONE INSIDE ETHEREUM BLOCKCHAIN))
NOTE : DON'T BE CONFUSED BETWEEN WETH AND ETHER, WETH IS ERC20 TOKEN VERSION OF ETHER. 
THEREFORE, WETH CAN BE FOUND IN ERC 20 TXN AND ETHER CAN BE FOUND IN INTERNAL TXN.
THE VALUE OF WETH AND ETH IS SAME IN TERMS OF USD.
"""

'\nEVERY ETHEREUM TXN DETAILS CAN BE FOUND WITHIN:\na) ALL TXN (TXN SEND "BY PERSON OR TO PERSON" TO THE ETHEREUM BLOCKCHAIN )\nb) INTERNAL TXN (ALL ETHER TXN (ETHEREUM NATIVE CURRENCY))\nc) ERC20 TXN  (ERC20 TOKEN TXN (DIVISIBLE TOKEN, COULD BE GENERATED BY ANYONE INSIDE ETHEREUM BLOCKCHAIN))\nNOTE : DON\'T BE CONFUSED BETWEEN WETH AND ETHER, WETH IS ERC20 TOKEN VERSION OF ETHER. \nTHEREFORE, WETH CAN BE FOUND IN ERC 20 TXN AND ETHER CAN BE FOUND IN INTERNAL TXN.\nTHE VALUE OF WETH AND ETH IS SAME IN TERMS OF USD.\n'

In [23]:
"""
TERMINOLOGY :
- GAS : GAS USED FOR A TXN (GAS UNITS * GAS PRICE) [THE VALUE CALCULATED THROUGHOUT IS IN TERMS OF ETHERS]
- VALUE : THE AMOUNT USED TO BUY/SELL IN TERMS OF ETHERS 
- TXN SEND BY PERSON OR RECEIVED BY THE PERSON : ALL TXN
- TRANSFER OF ETHERS FROM THE DESIRED ACCOUNT : INTERNAL TXN
- TRANSFER OF ERC20 TOKENS FROM THE DESIRED ACCOUNT : ERC20 TXN
- DESIRED ADDRESS : ETHEREUM ADDRESS ON WHICH ANALYSIS IS DONE
"""

'\nTERMINOLOGY :\n- GAS : GAS USED FOR A TXN (GAS UNITS * GAS PRICE) [THE VALUE CALCULATED THROUGHOUT IS IN TERMS OF ETHERS]\n- VALUE : THE AMOUNT USED TO BUY/SELL IN TERMS OF ETHERS \n- TXN SEND BY PERSON OR RECEIVED BY THE PERSON : ALL TXN\n- TRANSFER OF ETHERS FROM THE DESIRED ACCOUNT : INTERNAL TXN\n- TRANSFER OF ERC20 TOKENS FROM THE DESIRED ACCOUNT : ERC20 TXN\n- DESIRED ADDRESS : ETHEREUM ADDRESS ON WHICH ANALYSIS IS DONE\n'

In [24]:
"""
NOTE : WE HAVE FILTERED UNWANTED METHODS LIKE STAKING, NFT BUY BEFORE THE FOLLOWING DETAILS, STILL WE ARE PRONE TO SOME
UNWANTED METHODS (WHICH WE WILL BE ADDED IN DOUBTFULL TXN DICTIONARY)

FLOWCHART:
WE CHECK ALL TRANSACTION IN ALL TXN DICTIONARY ONE BY ONE
CHECK ALL TXN IF IT HAS NON-ZERO VALUE (THE TXN IS FOR BUY), ALL THE ETHERS SENT IN TXN MAY NOT BE USED SO WE CHECK INTERNAL FOR ANY UNSUSED ETHERS.

"""

'\nNOTE : WE HAVE FILTERED UNWANTED METHODS LIKE STAKING, NFT BUY BEFORE THE FOLLOWING DETAILS, STILL WE ARE PRONE TO SOME\nUNWANTED METHODS (WHICH WE WILL BE ADDED IN DOUBTFULL TXN DICTIONARY)\n\nFLOWCHART:\nWE CHECK ALL TRANSACTION IN ALL TXN DICTIONARY ONE BY ONE\nCHECK ALL TXN IF IT HAS NON-ZERO VALUE (THE TXN IS FOR BUY), ALL THE ETHERS SENT IN TXN MAY NOT BE USED SO WE CHECK INTERNAL FOR ANY UNSUSED ETHERS.\n\n'

In [61]:
#Retrieve ERC Transfer, Internal Txn, All Txn Data
def update_lists(address):
    
    txlist_api = 'https://api.etherscan.io/api?module=account&action=txlist&address={}&sort=asc&apikey=JV6FAU5SY7FDUB8AMI1VHXKS6QIASF89DC'.format(address)
    all_transactions = requests.request("GET", txlist_api)
    all_transactions_of_address = json.loads(all_transactions.text)['result']

    while len(all_transactions_of_address) > 9999 and len(all_transactions_of_address) < 50000:
        txlist_api_repeat = 'https://api.etherscan.io/api?module=account&action=txlist&startblock='+ all_transactions_of_address[-1]['blockNumber'] +'&sort=asc&address={}&apikey=JV6FAU5SY7FDUB8AMI1VHXKS6QIASF89DC'.format(address)
        all_transaction_repeat = requests.request("GET", txlist_api_repeat)
        result = json.loads(all_transaction_repeat.text)['result'][1:]

        all_transactions_of_address = all_transactions_of_address + result
        if len(result) < 9999:
            break

    erc_api = 'https://api.etherscan.io/api?module=account&action=tokentx&address={}&page=1&offset=10000&sort=asc&apikey=FI33TTAM3S1WVTGG5JDDMNTI61SPQ2GTUY '.format(address)
    erc_transfers = requests.request("GET", erc_api)
    all_erc_20_transfers = json.loads(erc_transfers.text)['result']


    while len(all_erc_20_transfers) > 9999 and len(all_erc_20_transfers) < 50000:
        txlist_api_repeat = 'https://api.etherscan.io/api?module=account&action=tokentx&startblock='+ all_erc_20_transfers[-1]['blockNumber'] +'&sort=asc&address={}&apikey=FI33TTAM3S1WVTGG5JDDMNTI61SPQ2GTUY'.format(address)
        all_transaction_repeat = requests.request("GET", txlist_api_repeat)
        result = json.loads(all_transaction_repeat.text)['result'][1:]

        all_erc_20_transfers = all_erc_20_transfers + result
        if len(result) < 9999:
            break

    txlist_internal_api = 'https://api.etherscan.io/api?module=account&action=txlistinternal&address={}&page=1&offset=10000&sort=asc&apikey=TRT149Y52NFS2DYR6HPJF3QW2P3DBTTAJ2'.format(address)
    internal_transactions = requests.request("GET", txlist_internal_api)
    all_internal_transactions = json.loads(internal_transactions.text)['result']

    while len(all_internal_transactions) > 9999 and len(all_internal_transactions) < 50000:
        txlist_api_repeat = 'https://api.etherscan.io/api?module=account&action=txlistinternal&startblock='+ all_internal_transactions[-1]['blockNumber'] +'&sort=asc&address={}&apikey=TRT149Y52NFS2DYR6HPJF3QW2P3DBTTAJ2'.format(address)
        all_transaction_repeat = requests.request("GET", txlist_api_repeat)
        result = json.loads(all_transaction_repeat.text)['result'][1:]

        all_internal_transactions = all_internal_transactions + result
        if len(result) < 9999:
            break
    return all_transactions_of_address, all_internal_transactions, all_erc_20_transfers


In [77]:
def get_ether_price(time_stamp):
    """
    RETURNS ETHER PRICE FOR A PARTICULAR DAY FROM TIMESTAMP, 
    IF DAILY PRICE IS NOT UPDATED, RETURNS PRICE OF LAST DAY FROM THE LIST
    """
    index = len(historic_ether_price['snapped_at'])-2
    date = str(datetime.fromtimestamp(int(time_stamp))).split()[0]
    for i,timestamp in enumerate(historic_ether_price['snapped_at']):
        if timestamp.split()[0] == date:
            index = i
            break
    if index == len(historic_ether_price['snapped_at'])-2 and date != historic_ether_price['snapped_at'][index].split()[0]:
        print("ether price error ::: " + date)
    return historic_ether_price['price'][index]

def convert_usd_to_eth(value, timestamp):
    ether_price = get_ether_price(timestamp)
    return value/ether_price

def calculate_gas(all_dict):
    return int(all_dict['all']['gasPrice'])*int(all_dict['all']['gasUsed'])/10**18

def filter_addresses_using_contracts(all_transactions_of_address):
    function_name_is_null = 0
    function_name_is_non_null = 0
    for transaction in all_transactions_of_address:
        if (transaction['from'] == address) and (transaction['functionName'] not in excluded_functionName) and (transaction['input'] not in excluded_input) and (transaction['isError'] != '1'):
            if transaction['functionName'] == '':
                function_name_is_null += 1
            else:
                function_name_is_non_null += 1
    if (function_name_is_non_null + function_name_is_null) == 0 or function_name_is_null/(function_name_is_non_null + function_name_is_null) > 0.3:
        address_using_contracts.append(address)
        return True
    else:
        return False

def is_reliable_token(contract_address):
    return contract_address in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                                '0xdac17f958d2ee523a2206206994597c13d831ec7',
                                '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                '0x6b175474e89094c44da98b954eedeac495271d0f']
  
def is_usd_token(contract_address):
    return contract_address in ['0xdac17f958d2ee523a2206206994597c13d831ec7',
                                '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                '0x6b175474e89094c44da98b954eedeac495271d0f']

def get_internal_txn_details_for_nonzero_value(address, all_dict):
    """
    UPDATES TOKEN TRANSACTION DETAILS FROM INTERNAL TXNS OR RETURNS TRUE IF TXN DETAILS NOT AVAILABLE IN INTERNAL TXNS
    ALSO THE FUNCTIONS UPDATES DOUBTFULL TXN DICTIONARY(FOR TXNS WE ARE NOT SURE IF WE WANT)
    NOTE: PLEASE CHECK DOUBTFULL TXN TO CHECK IF THE ETHEREUM ADDRESS IS NOT USING ANY UNKNOWN METHODS FOR BUY AND SELL.
    """
    global doubtfull_transactions
    global ethers_and_usd_conversions
    global hashwise_buy_sell_details

    _ether_decimals = 10**18
    value = int(all_dict['all']['value'])/_ether_decimals
    gas = calculate_gas(all_dict)

    #Subtracts unused ethers 
    if all_dict['internal']:
        value = (int(all_dict['all']['value']) - int(all_dict['internal']['value']))/_ether_decimals
    
    #This block is for later debugging (if any desired txn slips out), also we want to exit the fn without adding this txn to our pnl
    if not all_dict['internal']: 
        if (not all_dict['erc']):
            doubtfull_transactions['ether transfer/token purchased & transfer/other activity(nft purchase/staking)'].append(all_dict['all']['hash'])
        elif (is_reliable_token(all_dict['erc'][0]['contractAddress'])):     
            ethers_and_usd_conversions['ethers to usd txn'].append(all_dict['all']['hash'])
        else:
            doubtfull_transactions['miscelleneous doubtfull transactions'].append(all_dict['all']['hash'])
        return True
    
    hashwise_buy_sell_details[all_dict['all']['hash']] = ['buy', value, gas, all_dict['all']['timeStamp']]
   
def get_internal_txn_details_for_zero_value(address, all_dict):
    """
    UPDATES TOKEN TRANSACTION DETAILS FROM INTERNAL TXNS OR RETURNS TRUE IF TXN DETAILS IS NOT RELEVANT IN INTERNAL TXNS
    """
    global hashwise_buy_sell_details
    global sell_txn_of_transferred_tokens
    global ethers_and_usd_conversions

    if all_dict['internal'] and all_dict['internal']['value'] != '0':
        #To avoid selling USDC/USDT/DAI in our system
        if (all_dict['erc']) and (is_usd_token(all_dict['erc'][0]['contractAddress'])):
            ethers_and_usd_conversions['usd to ethers txn'].append(all_dict['all']['hash'])
            return True
        value = int(str(all_dict['internal']['value']))/(10**(18))
        gas = calculate_gas(all_dict)
        #Check if the tokens were not received
        txn_hash = all_dict['all']['hash']
        if all_dict['erc']: #To remove error of None ERC
            if all_dict['erc'][0]['contractAddress'] not in received_tokens_dict:
                hashwise_buy_sell_details[txn_hash] = ['sell', value, gas, all_dict['all']['timeStamp']]
            else:
                token_sender_address = received_tokens_dict[transaction['contractAddress']]
                sell_txn_of_transferred_tokens[token_sender_address][txn_hash] = ['sell', value, gas, transaction['timeStamp']]
    
    elif not all_dict['erc']:
        # All None, Internal None, ERC None (approve type txn), message txn (we don't want this txn)
        return True
    else:
        # All None, Internal None, ERC not None
        return False

def check_buy_txn_erc20(address, all_dict):
    global hashwise_buy_sell_details
    gas = calculate_gas(all_dict)
    
    for transaction in all_dict['erc']:
        if (is_reliable_token(transaction['contractAddress']) and transaction['from'] == address):
            value = int(str(transaction['value']))/(10**int(str(transaction['tokenDecimal'])))
            # if value not ether, convert to ether
            if transaction['contractAddress'] not in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']:
                value = convert_usd_to_eth(value,transaction['timeStamp'])
            hashwise_buy_sell_details[transaction['hash']] = ['buy', value, gas, transaction['timeStamp']]
            return True
    return False


def get_transaction_details_from_erc20(address, all_dict):
    """
    RETURNS TOKEN TRANSACTION DETAILS FROM ERC TXNS OR RETURNS TRUE IF TXN DETAILS NOT AVAILABLE IN ERC TXNS
    AIM : TO FIND SELL TXN WHERE TOKENS ARE CONVERTED TO WETH (WHICH IS ALSO A ERC20 TOKEN) OR TO GET BUY EVENT
    CASES : USD TO TOKEN, TOKEN TO USD, WETH TO TOKEN, TOKEN TO WETH
    """
    global hashwise_buy_sell_details
    global received_tokens_dict
    global sell_txn_of_transferred_tokens

    gas = calculate_gas(all_dict)
    #This txn would have been filtered if it was below is_valid_txn block
    if len(all_dict['erc']) == 1 and is_reliable_token(all_dict['erc'][0]['contractAddress']) and all_dict['erc'][0]['to'] in router_you_want_to_track_txn_for:
        value = int(str(all_dict['erc'][0]['value']))/(10**int(str(all_dict['erc'][0]['tokenDecimal'])))
        
        if all_dict['erc'][0]['contractAddress'] not in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']:
            value = convert_usd_to_eth(value,all_dict['erc'][0]['timeStamp'])
        hashwise_buy_sell_details[all_dict['erc'][0]['hash']] = ['buy', value, gas, transaction['timeStamp']]
        return None

    is_valid_txn = False
    # Filter weth or usd token transfers or usd to usd swaps (eg : BUSD to USDC swaps)
    for transaction in all_dict['erc']:
        if not is_reliable_token(transaction['contractAddress']):
            is_valid_txn = True
            break
    
    if not is_valid_txn:
        return None
    
    #Bought and kept the tokens
    if check_buy_txn_erc20(address, all_dict):
        return None

    #token_transfer from or to the account please add the complex txn list here
    if len(all_dict['erc']) == 1 and all_dict['erc'][0]['to'] == address and all_dict['erc'][0]['from'] not in router_you_want_to_track_txn_for:
        received_tokens_dict[all_dict['erc'][0]['contractAddress']] = all_dict['erc'][0]['from']
        return None
    
    #We have filtered buy txn, transfers, usd to usd swaps, transfer(len == 1 txn) upto this point

    able_to_get_final_sell_value = False
    for transaction in all_dict['erc']:
        
        if is_reliable_token(transaction['contractAddress']):
            value = int(str(transaction['value']))/(10**int(str(transaction['tokenDecimal'])))

            if transaction['contractAddress'] not in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']:
                value = convert_usd_to_eth(value,transaction['timeStamp'])

            if transaction['to'] == address.lower():
                able_to_get_final_sell_value = True
    
                if transaction['contractAddress'] not in received_tokens_dict:
                    hashwise_buy_sell_details[transaction['hash']] = ['sell', value, gas, transaction['timeStamp']]
                else:
                    token_sender_address = received_tokens_dict[transaction['contractAddress']]
                    sell_txn_of_transferred_tokens[token_sender_address][transaction['hash']] = ['sell', value, gas, transaction['timeStamp']]
    
    if able_to_get_final_sell_value == False:
        print("Another Unknown Doubt", all_dict['all']['hash'])

In [108]:
excluded_functionName = ['approve(address _spender, uint256 _value)',
                         'deposit()',
                         'setAddr(bytes32 node, uint256 coinType, bytes a)',
                         'setApprovalForAll(address operator, bool approved)',
                         'setName(string name)',
                         'withdraw(uint256 wad)',
                         'approveMax(address token)',
                         'getApprovalType(address token,uint256 amount)',
                         'mintDongz(uint256 _quantity)',
                         'registerProxy()',
                         'withdraw(uint256 amount)',
                         'claim()',
                         'withdrawERC20For(uint256 _withdrawalId, address _user, address _token, uint256 _amount, bytes _signatures)',
                         'atInversebrah(int248 a, uint48[] b, uint32 c, bytes20[] d, bytes30[] e)',
                         'mint()',
                         'depositETH(uint32 _l2Gas, bytes _data)',
                         'safeTransferFrom(address _from, address _to, uint256 _id, uint256 _value, bytes _data)',
                         'claimTokens(uint256 amount, bytes32[] merkleProof)',
                         'migrateTokens(address sourceToken, address toAddress, uint256 amount)',
                         'contribute()',
                         'unlock(address _token, address _recipient)',
                         'atomicMatch_(address[14] addrs, uint256[18] uints, uint8[8] feeMethodsSidesKindsHowToCalls, bytes calldataBuy, bytes calldataSell, bytes replacementPatternBuy, bytes replacementPatternSell, bytes staticExtradataBuy, bytes staticExtradataSell, uint8[2] vs, bytes32[5] rssMetadata)',
                         'purchase(uint256 _price)',
                         'migrateTokens(address sourceToken, address toAddress, uint256 amount)',
                         'claim(address _account, uint256[9] _data, bytes32[] _proof)',
                         'setRelayerApproval(address sender, address relayer, bool approved)',
                         'setApprovalForAll(address operator, bool authorized)',
                         'proxyAssert(address dest, uint8 howToCall, bytes calldata)',
                         'mintPublicSale(uint256 tokensCount)',
                         'deposit(uint256 _amount, uint256 _duration, address _receiver)',
                        ]

excluded_input = ['0x']
router_you_want_to_track_txn_for = ['0x7a250d5630b4cf539739df2c5dacb4c659f2488d','0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45', '0x1111111254fb6c44bAC0beD2854e76F90643097d'.lower(), '0xf18c3aad061fd9a737207d8d36751dcdcce25fdd'.lower(), '0xbace0cf4814a2390b59bfa2cbf4df9c11fb951d7'.lower(), '0x8c4b2115ff81f1bc4faad1d7745be689d540d1bb'.lower()]


In [109]:
all_address_transaction_details = {'address': 'hashwise_buy_sell_details'}
sell_txn_of_transferred_tokens  = {'address':{'hash':['sell','value','gas','date-timeStamp']}}
address_using_contracts = []

In [115]:
address_list = [x.lower() for x in [
'0x7d29c4f95c009ce4b158f68f06d56f02f460c9bb']]

In [116]:
for address in address_list:
    print(address)

    hashwise_buy_sell_details = {'hash': ['buy/sell','value','gas','date-timeStamp']}
    received_tokens_dict = {'contractAddress':'address from which token came'}
    hashwise_erc20_transaction = {'hash': ['{transactions}']}
    hashwise_all_transaction_data = {'hash': {'all':{}, 'internal':{}, 'erc':[]}}
    ethers_and_usd_conversions = {'ethers to usd txn': [], 
                                  'usd to ethers txn': []}
    doubtfull_transactions = {'ether transfer/token purchased & transfer/other activity(nft purchase/staking)': [],
                              'miscelleneous doubtfull transactions' : [],
                              'value is empty string' : []}
    all_transactions_of_address, all_internal_transactions, all_erc_20_transfers = update_lists(address)

    for transaction in all_erc_20_transfers:
        hashwise_erc20_transaction.setdefault(transaction['hash'], []).append(transaction)

    if filter_addresses_using_contracts(all_transactions_of_address):
        #These accounts uses contracts to trade(different calculations for them)
        break

    for transaction in all_transactions_of_address:      
        if (transaction['functionName'] not in excluded_functionName) and (transaction['input'] not in excluded_input) and (transaction['isError'] != '1'):
            internal, erc = None, None
            for internal_transaction in all_internal_transactions:
                if internal_transaction['hash'] == transaction['hash'] and internal_transaction['isError'] != '1':
                    internal = internal_transaction
                    break
            try:
                erc = hashwise_erc20_transaction[transaction['hash']]
            except:
                None #No erc txn available

            hashwise_all_transaction_data[transaction['hash']] = {'all': transaction, 'internal': internal, 'erc': erc}
    for transaction in hashwise_all_transaction_data:
        transaction_dict = hashwise_all_transaction_data[transaction]
        if not (transaction == 'hash'):
            if transaction_dict['all']['value'] == '':
                doubtfull_transactions['value is empty string'].append(transaction_dict['all']['hash'])
            elif transaction_dict['all']['value'] != '0':
                get_internal_txn_details_for_nonzero_value(address, transaction_dict)
            else:
                result = get_internal_txn_details_for_zero_value(address, transaction_dict)
                if result == False:
                    get_transaction_details_from_erc20(address, transaction_dict)

    all_address_transaction_details[address] = hashwise_buy_sell_details


0x7d29c4f95c009ce4b158f68f06d56f02f460c9bb
Another Unknown Doubt 0x54cf508bdc9f0e05a7828d4285d781950187f2d06ef4df8153c3428a0e7f5823
Another Unknown Doubt 0x4fae15fb3eb81233607636658d75d121c4997a310002422f1dce7bc3fb9f2726
Another Unknown Doubt 0xf8d8cea885fcace85d945e8df278e2e55e0c42cdcf1210f9f911174ae4c640e8
Another Unknown Doubt 0xb402c0a1da0e32ba4dcd057ed04a520ec1a5e5545bec5e46e869a10917a0ce33
Another Unknown Doubt 0xff443a3d5423c9f32916e66b038aefbbb107b351733e2799a467d5607f9f58d9


In [118]:
doubtfull_transactions

{'ether transfer/token purchased & transfer/other activity(nft purchase/staking)': [],
 'miscelleneous doubtfull transactions': ['0x59083bdbd05a6e948d19fa6c94587f9736a9842e2715e0a5d6500969183bb370'],
 'value is empty string': []}

In [119]:
ethers_and_usd_conversions

{'ethers to usd txn': ['0x486ec2b78dacee3d3ae872d65be957ddf39ac2a89884ae3844f7b21e94e5086c',
  '0x680a5537f2cd7fbab48271c5000955a9c839f4fc7a9bcde9144e3cfba7b5dcbf',
  '0x10c356293d81cd2483f957720f7f690c72460c92a785eafbf437ea30ce96cbb6',
  '0x53c9f7c3cc5133378f567755c78a6d4623fd07f7b9b568171fc452cdf8b60402',
  '0xb130013087b9c698d19fe8df725ecebf71d3a9773759afd0244d9fd4c7937019',
  '0x4d522af267697868211e32a9f4daaa34508a3f32d5780aa95db9ea7dbdc95f9d',
  '0x8d6d6a7cd6dce30be384b044acc9dcd26ae4a84abc5e50c98d12a9227ab04c32',
  '0xd8d1077905b6d9d1bd84a49d86b8eb59ca1084b5733c2c1f116aecd68c3f2479',
  '0x1d29f2051dda894ae5d50a8fb25ce8fee41dbe40603ad829d8484fe6ef296cc6',
  '0xa35c81280a7c3fb67d180ac1f970044e049145b2ffa6805ab092b7c2704615f8',
  '0x9e1dcd130fd53b4de06b1064e71936388f0806a6129810dc29334a217c3734fd',
  '0xd498afaa7324b81b7a097fe0dbafd3af8be557ebfff972e47cbb6b1e9022e547',
  '0xe1cce4c89780fbf3518a41354320c86ab87cb07a5133abe5055fec490f762975',
  '0x14f6d037ea860a68e17e850d55487c516f20a4

In [120]:
received_tokens_dict

{'contractAddress': 'address from which token came'}

In [121]:
# summary time stamp wise
start_timestamp = int(datetime(2021, 12, 1).timestamp())
end_timestamp = int(datetime(2023, 12, 30).timestamp())

time_stamp_wise_PandL = {'address':{'profit':'profit','gas':'gas'}}

for address in all_address_transaction_details:
    if address not in address_using_contracts:
        profit = 0
        gas = 0
        tnx_map = all_address_transaction_details[address]
        for each_individual_transaction in tnx_map:
            if not (each_individual_transaction == 'hash') and not (tnx_map == 'hashwise_buy_sell_details'):
                transaction = tnx_map[each_individual_transaction]
                time_stamp = int(transaction[3])

                if time_stamp > start_timestamp and time_stamp < end_timestamp:
                    gas = gas + transaction[2]
                    if transaction[0] == 'buy':
                        profit = profit - (transaction[1])
                    else:
                        profit = profit + (0.7)*(transaction[1])

        time_stamp_wise_PandL[address] = {'profit':profit, 'gas':gas}
        print(time_stamp_wise_PandL)

{'address': {'profit': 0, 'gas': 0}}
{'address': {'profit': 0, 'gas': 0}, '0xa10e3af40d3a4ebcb7406cbf3eb16c1a0acb1888': {'profit': 131.6123807106264, 'gas': 18.243232325166264}}
{'address': {'profit': 0, 'gas': 0}, '0xa10e3af40d3a4ebcb7406cbf3eb16c1a0acb1888': {'profit': 131.6123807106264, 'gas': 18.243232325166264}, '0x7d29c4f95c009ce4b158f68f06d56f02f460c9bb': {'profit': 196.2403584061261, 'gas': 32.74365509886352}}
